<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания №19


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

Создать базовый класс Subscription в C#, который будет представлять подписки на
различные услуги. На основе этого класса разработать 2-3 производных класса,
демонстрирующих принципы наследования и полиморфизма. В каждом из классов
должны быть реализованы новые атрибуты и методы, а также переопределены
некоторые методы базового класса для демонстрации полиморфизма.

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) создайте явную реализации интерфейса и управление зависимостями 


<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [ ]:
using System;
using System.Collections.Generic;
using System.Linq; 

// шаг 1: добавляем новые интерфейсы для демонстрации

// интерфейс для любого объекта, который можно логировать
public interface ILoggable
{
    void LogStatus();
}

// основной интерфейс для всех подписок, будет использоваться для управления зависимостями
public interface ISubscription
{
    string ServiceName { get; }
    double Cost { get; }
    string GetSubscriptionDetails();
}


// интерфейс для подписок, к которым можно применить промокод
public interface IPromoCodeApplicable
{
    // метод для применения промокода
    void ApplyPromoCode(string promoCode);
}

// интерфейс для сервисов, имеющих профиль пользователя
public interface IUserProfile
{
    // свойство для имени пользователя
    string UserName { get; set; }
    // метод для обновления профиля
    void UpdateProfile(string newName);
}


// базовый класс для подписок, теперь реализует ISubscription
public class Subscription : ISubscription
{
    public int SubscriptionId { get; set; }
    public string ServiceName { get; set; }
    public double Cost { get; set; }
    
    public DateTime StartDate { get; private set; } // дата начала подписки
    public bool IsActive { get; private set; }      // статус активности
    
    // новый атрибут
    public string BillingCycle { get; set; } // цикл оплаты (ежемесячно, ежегодно)

    public Subscription(int id, string serviceName, double cost)
    {
        SubscriptionId = id;
        ServiceName = serviceName;
        Cost = cost;
        StartDate = DateTime.Now; 
        IsActive = true;
        BillingCycle = "Ежемесячно"; // значение по умолчанию
    }

    public void ApplyDiscountFrom(Subscription otherSubscription, double discountPercentage)
    {
        double discountAmount = otherSubscription.Cost * (discountPercentage / 100);
        this.Cost -= discountAmount;
        Console.WriteLine($"Применена скидка {discountPercentage}% от подписки '{otherSubscription.ServiceName}'. Новая стоимость '{this.ServiceName}': {this.Cost:C}");
    }
    
    public void CancelSubscription()
    {
        IsActive = false;
        Console.WriteLine($"Подписка на '{ServiceName}' была отменена");
    }
    
    // новый метод
    public void Reactivate()
    {
        IsActive = true;
        Console.WriteLine($"Подписка на '{ServiceName}' снова активна.");
    }
    
    public int GetSubscriptionAgeInDays()
    {
        return (int)(DateTime.Now - StartDate).TotalDays;
    }

    public virtual double CalculateMonthlyCost()
    {
        return Cost;
    }

    public virtual void ExtendSubscription(int months)
    {
        Console.WriteLine($"Подписка на '{ServiceName}' продлена на {months} мес.");
    }
    
    public void ExtendSubscription()
    {
        ExtendSubscription(1); 
    }
    
    public void ExtendSubscription(int months, double bonusDiscount)
    {
        Console.WriteLine($"Подписка на '{ServiceName}' продлена на {months} мес. с дополнительной скидкой {bonusDiscount}%!");
    }


    public virtual string GetSubscriptionDetails()
    {
        return $"ID: {SubscriptionId}, Услуга: {ServiceName}, Стоимость: {Cost:C} ({BillingCycle}), Активна: {IsActive}";
    }
}

// класс реализует несколько интерфейсов, включая ILoggable
public class OnlineServiceSubscription : Subscription, IUserProfile, IPromoCodeApplicable, ILoggable
{
    public int MaxUsers { get; set; }
    public double UsedStorageGB { get; set; } 
    public string UserName { get; set; } 
    
    // новый атрибут
    public string ServiceRegion { get; set; } // регион серверов

    public OnlineServiceSubscription(int id, string serviceName, double costPerUser, int maxUsers, string userName)
        : base(id, serviceName, costPerUser)
    {
        MaxUsers = maxUsers;
        UserName = userName;
        UsedStorageGB = 0; 
        ServiceRegion = "EU-West"; // значение по умолчанию
    }
    
    public void CheckStorageUsage()
    {
        Console.WriteLine($"Использование хранилища: {UsedStorageGB} GB");
    }
    
    // новый метод
    public void ChangeRegion(string newRegion)
    {
        ServiceRegion = newRegion;
        Console.WriteLine($"Регион сервиса изменен на {ServiceRegion}");
    }

    public void UpdateProfile(string newName)
    {
        Console.WriteLine($"Имя пользователя для '{ServiceName}' изменено с '{UserName}' на '{newName}'");
        UserName = newName;
    }

    public void ApplyPromoCode(string promoCode)
    {
        if (promoCode == "CLOUD20")
        {
            Cost *= 0.8; 
            Console.WriteLine($"Промокод 'CLOUD20' применен! Новая базовая стоимость: {Cost:C}");
        }
        else
        {
            Console.WriteLine("Неверный промокод");
        }
    }

    public override double CalculateMonthlyCost()
    {
        return Cost * MaxUsers;
    }
    
    // шаг 2: Создание явной реализации интерфейса
    // метод скрыт от прямого доступа через экземпляр класса
    // его можно вызвать, только если объект приведен к типу ILoggable
    void ILoggable.LogStatus()
    {
        Console.WriteLine($"[ЛОГ]: Статус сервиса '{ServiceName}': Активен, Пользователь: {UserName}, Хранилище: {UsedStorageGB} GB.");
    }
}

public class StreamingSubscription : Subscription, IPromoCodeApplicable
{
    public int MaxStreams { get; set; }
    public bool HasOfflineMode { get; set; } 

    public StreamingSubscription(int id, string serviceName, double cost, int maxStreams, bool hasOfflineMode)
        : base(id, serviceName, cost)
    {
        MaxStreams = maxStreams;
        HasOfflineMode = hasOfflineMode;
    }
    
    public void ToggleOfflineMode()
    {
        HasOfflineMode = !HasOfflineMode;
        Console.WriteLine($"Офлайн-режим для '{ServiceName}' теперь {(HasOfflineMode ? "включен" : "выключен")}");
    }
    
    public void ApplyPromoCode(string promoCode)
    {
        if (promoCode == "MUSIC15")
        {
            Cost *= 0.85; 
            Console.WriteLine($"Промокод 'MUSIC15' применен! Новая базовая стоимость: {Cost:C}");
        }
        else
        {
            Console.WriteLine("Неверный промокод");
        }
    }

    public override void ExtendSubscription(int months)
    {
        Console.WriteLine($"Подписка на '{ServiceName}' продлена на {months} мес. Специальное предложение: +1 месяц в подарок!");
    }
}

public class VideoSubscription : Subscription
{
    public string VideoQuality { get; set; }
    public List<string> IncludedChannels { get; set; }

    public VideoSubscription(int id, string serviceName, double cost, string videoQuality)
        : base(id, serviceName, cost)
    {
        VideoQuality = videoQuality;
        IncludedChannels = new List<string>(); 
    }
    
    public void DisplayChannels()
    {
        Console.WriteLine("Включенные каналы:");
        foreach (var channel in IncludedChannels)
        {
            Console.WriteLine($"- {channel}");
        }
    }

    public override string GetSubscriptionDetails()
    {
        return base.GetSubscriptionDetails() + $", Качество видео: {VideoQuality}";
    }
}

public class PremiumVideoSubscription : VideoSubscription
{
    public bool HasPersonalManager { get; set; } 

    public PremiumVideoSubscription(int id, string serviceName, double cost, string videoQuality, bool hasManager)
        : base(id, serviceName, cost, videoQuality) 
    {
        HasPersonalManager = hasManager;
    }
    
    public void ContactManager()
    {
        if (HasPersonalManager)
        {
            Console.WriteLine("Соединение с вашим персональным менеджером...");
        }
        else
        {
            Console.WriteLine("Эта услуга не включена в вашу подписку");
        }
    }

    public override string GetSubscriptionDetails()
    {
        return base.GetSubscriptionDetails() + $", Персональный менеджер: {(HasPersonalManager ? "Да" : "Нет")}";
    }
}

public class SubscriptionManager<T> where T : Subscription
{
    private List<T> _subscriptions = new List<T>();

    public void Add(T subscription)
    {
        _subscriptions.Add(subscription);
        Console.WriteLine($"В менеджер добавлена подписка: '{subscription.ServiceName}'");
    }

    public void DisplayAll()
    {
        Console.WriteLine($"\n--- Детали подписок в менеджере ({typeof(T).Name}) ---");
        if (!_subscriptions.Any())
        {
            Console.WriteLine("В менеджере нет подписок.");
            return;
        }
        
        foreach (var sub in _subscriptions)
        {
            Console.WriteLine(sub.GetSubscriptionDetails());
        }
        Console.WriteLine("-------------------------------------------");
    }
}

// --- Шаг 3: Создание класса для управления зависимостями ---

// этот сервис зависит от АБСТРАКЦИИ (ISubscription), а не от конкретного класса
public class BillingService
{
    private readonly ISubscription _subscriptionToProcess;

    // зависимость "внедряется" через конструктор (Dependency Injection)
    public BillingService(ISubscription subscription)
    {
        _subscriptionToProcess = subscription;
    }

    // метод, который использует зависимость
    public void ProcessMonthlyPayment()
    {
        Console.WriteLine($"\nОбработка платежа для сервиса '{_subscriptionToProcess.ServiceName}'...");
        Console.WriteLine($"С вашего счета будет списано: {_subscriptionToProcess.Cost:C}.");
        Console.WriteLine("Платеж успешно проведен.");
    }
}



// ----- ОСНОВНАЯ ПРОГРАММА -----

List<Subscription> subscriptions = new List<Subscription>();

var onlineStorage = new OnlineServiceSubscription(101, "Облачное хранилище", 5.0, 10, "User123");
var musicService = new StreamingSubscription(202, "Музыкальный сервис", 15.0, 4, true);
var premiumVideo = new PremiumVideoSubscription(404, "Премиум Кино", 40.0, "8K HDR", true);
premiumVideo.IncludedChannels.Add("Новости Кино");
premiumVideo.IncludedChannels.Add("Классика");

// ... [существующий код демонстрации опущен для краткости] ...
Console.WriteLine("... (пропуск основной демонстрации для краткости) ...");


// --- ДЕМОНСТРАЦИЯ ВЫПОЛНЕНИЯ ДОПОЛНИТЕЛЬНОГО ЗАДАНИЯ ---

Console.WriteLine("\n\n--- Демонстрация явной реализации интерфейса ---\n");

// метод LogStatus() недоступен напрямую
// onlineStorage.LogStatus(); // это вызовет ошибку компиляции

// чтобы вызвать метод, приводим объект к типу интерфейса
ILoggable logger = onlineStorage;
logger.LogStatus();

Console.WriteLine("\n--- Демонстрация управления зависимостями (Dependency Injection) ---\n");

// создаем экземпляр сервиса, "внедряя" в него конкретную подписку
BillingService musicBilling = new BillingService(musicService);
musicBilling.ProcessMonthlyPayment();

// тот же сервис может работать с другой подпиской без каких-либо изменений
BillingService videoBilling = new BillingService(premiumVideo);
videoBilling.ProcessMonthlyPayment();